# Using regular expressions to parse through a messy .txt file

#### Imports

In [1]:
#https://github.com/Austin-Czyzewski/Regular-Expressions-Demo
import glob
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set()
import pandas as pd

#### Grab our file

In [2]:
files = glob.glob("*.txt")
print(files)

['thisisacrazyfile.txt']


#### Open Our file with Python

In [3]:
with open(files[0], 'r') as file:
    giant_string = file.read()
    
print(giant_string)

lkjahsdkjfh$AAPL.{formatting__Garbage__January 11 2021}.newgarbage..youdontneedtoknowthisstuff::132.43dkjhasdf--Apple Inc.--alsdi;fasdf;liu ;oa
kasdfkj$F.{formatting__Garbage__January 8 2021}.newgarbage..youdontneedtoknowthisstuff::9.10dkjhasdf--Ford Motor Company--alsdi;fasdf;liu ;oa
asdfasvdkjhfglasjkdblf; jabn$AMZN.{formatting__Garbage__January 5 2021}.newgarbage..youdontneedtoknowthisstuff::3166.01dkjhasdf--Amazon.com, Inc.--alsdi;fasdf;liu ;oa
ahsd;f$GME.{formatting__Garbage__December 29 2020}.newgarbage..youdontneedtoknowthisstuff::20.82kjhasdf--GameStop Corp.--alsdi;fasdf;liu ;oa
$GOEV.{formatting__Garbage__January 7 2021}.newgarbage..youdontneedtoknowthisstuff::14.49dkjhasdf--Canoo Holdings Ltd.--alsdi;fasdf;liu ;oa
adlsifhlaisdhfklhalsdkjhflkjahsdlkjug askdfkagsdkuvg lakusdghfkuashldkiuvfh aksjdhflkajshd flkj hasf$F.{formatting__Garbage__December 10 2020}.newgarbage..youdontneedtoknowthisstuff::9.31dkjhasdf--Ford Motor Company--alsdi;fasdf;liu ;oa
-----------------------------

#### File breakdown:
 - You can see that we have 4 major pieces of information hidden in this file. The first is the ticker symbol. In this case it is always started with a '\$' and ends with a '.'
 - The next is the date, this is within curly brackets and is always at the end of those brackets following '__'
 - Next up is the opening price of that stock on the date. This is always following '::' and has text after it.
 - Finally, we have the name of the company, located within two sets of '--'... but we also have random breaks of many '-' that may throw a hitch in some of our plans.

In [4]:
ticker_symbols = re.findall('[$].{,5}[.]{1}',giant_string)
#What this regex is saying is:
    #We are looking for any instance of a literal $ followed by up to 5 characters (longest length of a ticker symbol) and then followed by one literal '.'
dates = re.findall('__.{,11}\s\d+[}]', giant_string)
#This regex says: we are looking for any instance of '__' followed by up to 11 characters (longest month (September, plus our two __) that precede a space.
    #That space must then be followed by a number and finally end with a bracket.
opening_prices = re.findall('::\d+\.\d+',giant_string)
#This regex searches for: two colons followed by any number of numbers followed by exactly one . and then any number of numbers (this could also be set to just two numbers)
company_names = re.findall('(?!-{3,})(--.+--)', giant_string)
#This regex includes any exclusion statement that says: do not include any information that is between 3 or more hyphens. If that condition is not met, then look for 
# Any characters between two sets of --

In [5]:
for number, (symbol, date, price, name) in enumerate(zip(ticker_symbols, dates, opening_prices, company_names)):
    print(number, symbol, date, price, name)

0 $AAPL. __January 11 2021} ::132.43 --Apple Inc.--
1 $F. __January 8 2021} ::9.10 --Ford Motor Company--
2 $AMZN. __January 5 2021} ::3166.01 --Amazon.com, Inc.--
3 $GME. __December 29 2020} ::20.82 --GameStop Corp.--
4 $GOEV. __January 7 2021} ::14.49 --Canoo Holdings Ltd.--
5 $F. __December 10 2020} ::9.31 --Ford Motor Company--
6 $AMZN. __January 11 2021} ::3180.00 --Amazon.com, Inc.--
7 $AAPL. __December 28 2020} ::133.99 --Apple Inc.--
8 $GME. __December 11 2020} ::13.91 --GameStop Corp.--
9 $TSLA. __January 5 2021} ::723.66 --Tesla, Inc.--
10 $AMZN. __January 7 2021} ::3157.00 --Amazon.com, Inc.--


#### Clean up our data a little bit

We know what we were searching for, so this step usually just includes getting rid of those pesky delimiters.

In [6]:
for number, (symbol, date, price, name) in enumerate(zip(ticker_symbols, dates, opening_prices, company_names)):
    ticker_symbols[number] = re.sub('\$|\.','',symbol) #Replace all literal $ or literal . with nothing
    dates[number] = re.sub('__|\}','',date) #Replace all __ or curly end brackets with nothing
    opening_prices[number] = float(re.sub('::','',price)) #You see the trend now, but this time we are casting that result as a float
    company_names[number] = re.sub('--','',name)

In [7]:
for number, (symbol, date, price, name) in enumerate(zip(ticker_symbols, dates, opening_prices, company_names)):
    print(number, symbol, date, price, name)

0 AAPL January 11 2021 132.43 Apple Inc.
1 F January 8 2021 9.1 Ford Motor Company
2 AMZN January 5 2021 3166.01 Amazon.com, Inc.
3 GME December 29 2020 20.82 GameStop Corp.
4 GOEV January 7 2021 14.49 Canoo Holdings Ltd.
5 F December 10 2020 9.31 Ford Motor Company
6 AMZN January 11 2021 3180.0 Amazon.com, Inc.
7 AAPL December 28 2020 133.99 Apple Inc.
8 GME December 11 2020 13.91 GameStop Corp.
9 TSLA January 5 2021 723.66 Tesla, Inc.
10 AMZN January 7 2021 3157.0 Amazon.com, Inc.


That looks much better, now that we have that sorted, we can push this into an excel file.

In [8]:
stock_data = pd.DataFrame([ticker_symbols, dates, opening_prices, company_names], index = ['Ticker Symbol','Date','Opening Price (USD)','Company Name'])
#First we will put our data into a dataframe to make use of a nifty Pandas demo
print(stock_data.head())
print('-'*60)
stock_data = stock_data.transpose()
#I prefer my data to expand into the rows, not the column
print(stock_data.head())

stock_data.to_excel('Stock_Data.xlsx', index = False) #Write the dataframe to excel without the index. Our index contains no unique information so we get rid of it

                                   0                   1                 2  \
Ticker Symbol                   AAPL                   F              AMZN   
Date                 January 11 2021      January 8 2021    January 5 2021   
Opening Price (USD)           132.43                 9.1           3166.01   
Company Name              Apple Inc.  Ford Motor Company  Amazon.com, Inc.   

                                    3                    4  \
Ticker Symbol                     GME                 GOEV   
Date                 December 29 2020       January 7 2021   
Opening Price (USD)             20.82                14.49   
Company Name           GameStop Corp.  Canoo Holdings Ltd.   

                                      5                 6                 7  \
Ticker Symbol                         F              AMZN              AAPL   
Date                   December 10 2020   January 11 2021  December 28 2020   
Opening Price (USD)                9.31              3180    